# Import the Libraries

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import os
import numpy as np

# Load Dataset

In [2]:
data=pd.read_csv('NIFTY50_all.csv')

In [3]:
data.head()

,Date,Symbol,Series,Prev Close,Open,High,Low,Last,Close,VWAP,Volume,Turnover,Trades,Deliverable Volume,%Deliverble
0,2007-11-27,MUNDRAPORT,EQ,440.00,770.00,1050.00,770.0,959.0,962.90,984.72,27294366,2.687719e+15,NaN,9859619.0,0.3612
1,2007-11-28,MUNDRAPORT,EQ,962.90,984.00,990.00,874.0,885.0,893.90,941.38,4581338,4.312765e+14,NaN,1453278.0,0.3172
2,2007-11-29,MUNDRAPORT,EQ,893.90,909.00,914.75,841.0,887.0,884.20,888.09,5124121,4.550658e+14,NaN,1069678.0,0.2088
3,2007-11-30,MUNDRAPORT,EQ,884.20,890.00,958.00,890.0,929.0,921.55,929.17,4609762,4.283257e+14,NaN,1260913.0,0.2735
4,2007-12-03,MUNDRAPORT,EQ,921.55,939.75,995.00,922.0,980.0,969.30,965.65,2977470,2.875200e+14,NaN,816123.0,0.2741


In [4]:
data['Date'] = pd.to_datetime(data['Date'])

In [5]:
data['Returns'] = data['Close'] / data['Close'].shift(1) - 1

In [6]:
data['YearMonth'] = data['Date'].dt.to_period('M')

In [7]:
monthly_volatility = data.groupby('YearMonth')['Returns'].std()

In [8]:
for month in range(1, 13):
    month_name = pd.Timestamp(year=2000, month=month, day=1).strftime('%B')
    print(f"Volatility for {month_name}: {monthly_volatility[pd.Timestamp(year=2000, month=month, day=1).to_period('M')]}")

Volatility for January: 0.8977689492081633
Volatility for February: 0.04513421310138912
Volatility for March: 0.05802692704382143
Volatility for April: 0.04728742322904806
Volatility for May: 0.045401401619042006
Volatility for June: 0.030700171479876963
Volatility for July: 0.0482993656283617
Volatility for August: 0.030415661197128966
Volatility for September: 0.030985231898859367
Volatility for October: 0.03060700379165393
Volatility for November: 0.03375172520425308
Volatility for December: 0.03496061106572765


# Volatility (2000-2021)

In [9]:
for year in range(2000, 2022):
    for month in range(1, 13):
        month_name = pd.Timestamp(year=year, month=month, day=1).strftime('%B')
        year_month = pd.Timestamp(year=year, month=month, day=1).to_period('M')
        print(f"Volatility for {month_name} {year}: {monthly_volatility.get(year_month, 'N/A')}")

Volatility for January 2000: 0.8977689492081633
Volatility for February 2000: 0.04513421310138912
Volatility for March 2000: 0.05802692704382143
Volatility for April 2000: 0.04728742322904806
Volatility for May 2000: 0.045401401619042006
Volatility for June 2000: 0.030700171479876963
Volatility for July 2000: 0.0482993656283617
Volatility for August 2000: 0.030415661197128966
Volatility for September 2000: 0.030985231898859367
Volatility for October 2000: 0.03060700379165393
Volatility for November 2000: 0.03375172520425308
Volatility for December 2000: 0.03496061106572765
Volatility for January 2001: 0.02834262640131242
Volatility for February 2001: 0.031146629699661678
Volatility for March 2001: 0.055750055277824534
Volatility for April 2001: 0.04726189014622288
Volatility for May 2001: 0.02612789953293664
Volatility for June 2001: 0.027936067389464806
Volatility for July 2001: 0.042166690624466034
Volatility for August 2001: 0.023696782287454032
Volatility for September 2001: 0.0443

In [20]:
#Threshold volitility t=v1+v2+v3....vn / number of months 

for year in range(2000, 2022):
    for month in range(1, 13):
        month_name = pd.Timestamp(year=year, month=month, day=1).strftime('%B')
        year_month = pd.Timestamp(year=year, month=month, day=1).to_period('M')
        volatility = {monthly_volatility.get(year_month, 'N/A')}

threshold_volatility = 0.04
result = []
volatilities = {monthly_volatility.get(year_month, 'N/A')}
        
for volatility in volatilities:
    # Check if the volatility is 'N/A'
    if volatility == 'N/A':
        result.append('N/A')
    else:
        # Convert the volatility to float and then compare
        if float(volatility) < threshold_volatility:
            result.append(0)
        else:
            result.append(1)
for i, value in enumerate(result, start=1):
    print(f"Volatility for Month {i}: {value}")

Volatility for Month 1: N/A


# Crash prediction using volatility

In [25]:
volatilities = [
    0.8977689492081633, 0.04513421310138912, 0.05802692704382143, 0.04728742322904806,
    0.045401401619042006, 0.030700171479876963, 0.0482993656283617, 0.030415661197128966,
    0.030985231898859367, 0.03060700379165393, 0.03375172520425308, 0.03496061106572765,
    0.02834262640131242, 0.031146629699661678, 0.055750055277824534, 0.04726189014622288,
    0.02612789953293664, 0.027936067389464806, 0.042166690624466034, 0.023696782287454032,
    0.044338399953617734, 0.03378340776049961, 0.029152671857196884, 0.029448222222873976,
    0.02821749608890883, 0.055111760603345133, 0.026806717585252948, 0.026772153259087585,
    0.02981315986554379, 0.02915761484463663, 0.025558251942487818, 0.01937807919177481,
    0.022067238093612326, 0.02242424860065373, 0.01993305756658776, 0.027601400875720924,
    0.02698360543297294, 0.017351376964464482, 0.01792970952965921, 0.026971355957663812,
    0.0240917240140225, 0.023687148877659138, 0.03946972716247293, 0.03330302291857,
    0.028470834110919874, 0.028766673650430136, 0.025937705246657797, 0.02717312983785784,
    0.04681464391355925, 0.02901306858683882, 0.026781406933802918, 0.02401084032096642,
    0.05981625624577183, 0.04064793101841395, 0.034454129690917885, 0.04068035960690894,
    0.016819077796566963, 0.019731217004208778, 0.039630135741278896, 0.019546208268417367,
    0.025737944361724236, 0.026890752381579393, 0.03731453604318986, 0.02069238103519333,
    0.017450736332063285, 0.016434655869671265, 0.01979086010166755, 0.04322487847645081,
    0.050075446093198916, 0.026810355908614615, 0.018819087763554364, 0.019312419592265675,
    0.021403147470162687, 0.021646913293074065, 0.022038212242964123, 0.03484121661912447,
    0.03553259420384605, 0.04359515612076832, 0.03636969419176993, 0.036748448117949806,
    0.025629035882919912, 0.022662821654742312, 0.01978190422475699, 0.02523161552916217,
    0.02240146749017488, 0.026817871893285692, 0.032030368049797225, 0.0249965109022357,
    0.019710287043970025, 0.018151189986873328, 0.020334556168468673, 0.02486655502369652,
    0.021228052987678436, 0.034037403902988164, 0.02772127911175314, 0.02614133968203964,
    0.04330021063736597, 0.03204664900906198, 0.03889276511974095, 0.025530972271072074,
    0.03923211571521944, 0.03047582719721922, 0.03854668113031836, 0.04055410981421861,
    0.028861618180955288, 0.0678200279809829, 0.043549329224859955, 0.03510928972981552,
    0.037635293928042005, 0.027149168782528318, 0.034400153317782664, 0.03721349571139837,
    0.05066473435889667, 0.03695187053789371, 0.03488358902577831, 0.02670192273776573,
    0.020995548627629413, 0.03035987871170744, 0.028414611847047813, 0.01748895568248525,
    0.029353070218025662, 0.02042835848683063, 0.022801588173391334, 0.018340034985742305,
    0.025570709405996477, 0.022889026487500793, 0.015265893197830982, 0.033578068752215126,
    0.044573945575188255, 0.017512025066901995, 0.04043602672409526, 0.01929890837510732,
    0.02070643703790456, 0.03357326962536876, 0.019661795842461055, 0.017215674083610733,
    0.01919396304067829, 0.033160930917451543, 0.029556707309713146, 0.02266638459260127,
    0.033123017825700686, 0.021920553473538825, 0.020826528659979204, 0.020882062829241835]
months = [
    "January", "February", "March", "April", "May", "June", "July", "August", "September", "October", "November", "December"
]
result = []


for i, volatility in enumerate(volatilities):
    # Check if the volatility is 'N/A'
    if volatility == 'N/A':
        result.append('N/A')
    else:
        # Convert the volatility to float and then compare
        if float(volatility) < threshold_volatility:
            result.append(0)
        else:
            result.append(1)

# Print the result for each month
for i, value in enumerate(result, start=1):
    year = 2000 + (i // 12)
    month = months[i % 12 - 1]
    print(f"Volatility for {month} {year}: {value}")

Volatility for January 2000: 1
Volatility for February 2000: 1
Volatility for March 2000: 1
Volatility for April 2000: 1
Volatility for May 2000: 1
Volatility for June 2000: 0
Volatility for July 2000: 1
Volatility for August 2000: 0
Volatility for September 2000: 0
Volatility for October 2000: 0
Volatility for November 2000: 0
Volatility for December 2001: 0
Volatility for January 2001: 0
Volatility for February 2001: 0
Volatility for March 2001: 1
Volatility for April 2001: 1
Volatility for May 2001: 0
Volatility for June 2001: 0
Volatility for July 2001: 1
Volatility for August 2001: 0
Volatility for September 2001: 1
Volatility for October 2001: 0
Volatility for November 2001: 0
Volatility for December 2002: 0
Volatility for January 2002: 0
Volatility for February 2002: 1
Volatility for March 2002: 0
Volatility for April 2002: 0
Volatility for May 2002: 0
Volatility for June 2002: 0
Volatility for July 2002: 0
Volatility for August 2002: 0
Volatility for September 2002: 0
Volatilit

In [26]:
import csv

data = [
    "Volatility for January 2000: 1",
    "Volatility for February 2000: 1",
    "Volatility for March 2000: 1",
    "Volatility for April 2000: 1",
    "Volatility for May 2000: 1",
    "Volatility for June 2000: 0",
    "Volatility for July 2000: 1",
    "Volatility for August 2000: 0",
    "Volatility for September 2000: 0",
    "Volatility for October 2000: 0",
    "Volatility for November 2000: 0",
    "Volatility for December 2001: 0",
    "Volatility for January 2001: 0",
    "Volatility for February 2001: 0",
    "Volatility for March 2001: 1",
    "Volatility for April 2001: 1",
    "Volatility for May 2001: 0",
    "Volatility for June 2001: 0",
    "Volatility for July 2001: 1",
    "Volatility for August 2001: 0",
    "Volatility for September 2001: 1",
    "Volatility for October 2001: 0",
    "Volatility for November 2001: 0",
    "Volatility for December 2002: 0",
    "Volatility for January 2002: 0",
    "Volatility for February 2002: 1",
    "Volatility for March 2002: 0",
    "Volatility for April 2002: 0",
    "Volatility for May 2002: 0",
    "Volatility for June 2002: 0",
    "Volatility for July 2002: 0",
    "Volatility for August 2002: 0",
    "Volatility for September 2002: 0",
    "Volatility for October 2002: 0",
    "Volatility for November 2002: 0",
    "Volatility for December 2003: 0",
    "Volatility for January 2003: 0",
    "Volatility for February 2003: 0",
    "Volatility for March 2003: 0",
    "Volatility for April 2003: 0",
    "Volatility for May 2003: 0",
    "Volatility for June 2003: 0",
    "Volatility for July 2003: 0",
    "Volatility for August 2003: 0",
    "Volatility for September 2003: 0",
    "Volatility for October 2003: 0",
    "Volatility for November 2003: 0",
    "Volatility for December 2004: 0",
    "Volatility for January 2004: 1",
    "Volatility for February 2004: 0",
    "Volatility for March 2004: 0",
    "Volatility for April 2004: 0",
    "Volatility for May 2004: 1",
    "Volatility for June 2004: 1",
    "Volatility for July 2004: 0",
    "Volatility for August 2004: 1",
    "Volatility for September 2004: 0",
    "Volatility for October 2004: 0",
    "Volatility for November 2004: 0",
    "Volatility for December 2005: 0",
    "Volatility for January 2005: 0",
    "Volatility for February 2005: 0",
    "Volatility for March 2005: 0",
    "Volatility for April 2005: 0",
    "Volatility for May 2005: 0",
    "Volatility for June 2005: 0",
    "Volatility for July 2005: 0",
    "Volatility for August 2005: 1",
    "Volatility for September 2005: 1",
    "Volatility for October 2005: 0",
    "Volatility for November 2005: 0",
    "Volatility for December 2006: 0",
    "Volatility for January 2006: 0",
    "Volatility for February 2006: 0",
    "Volatility for March 2006: 0",
    "Volatility for April 2006: 0",
    "Volatility for May 2006: 0",
    "Volatility for June 2006: 1",
    "Volatility for July 2006: 0",
    "Volatility for August 2006: 0",
    "Volatility for September 2006: 0",
    "Volatility for October 2006: 0",
    "Volatility for November 2006: 0",
    "Volatility for December 2007: 0",
    "Volatility for January 2007: 0",
    "Volatility for February 2007: 0",
    "Volatility for March 2007: 0",
    "Volatility for April 2007: 0",
    "Volatility for May 2007: 0",
    "Volatility for June 2007: 0",
    "Volatility for July 2007: 0",
    "Volatility for August 2007: 0",
    "Volatility for September 2007: 0",
    "Volatility for October 2007: 0",
    "Volatility for November 2007: 0",
    "Volatility for December 2008: 0",
    "Volatility for January 2008: 1",
    "Volatility for February 2008: 0",
    "Volatility for March 2008: 0",
    "Volatility for April 2008: 0",
    "Volatility for May 2008: 0",
    "Volatility for June 2008: 0",
    "Volatility for July 2008: 0",
    "Volatility for August 2008: 1",
    "Volatility for September 2008: 0",
    "Volatility for October 2008: 1",
    "Volatility for November 2008: 1",
    "Volatility for December 2009: 0",
    "Volatility for January 2009: 0",
    "Volatility for February 2009: 0",
    "Volatility for March 2009: 0",
    "Volatility for April 2009: 0",
    "Volatility for May 2009: 1",
    "Volatility for June 2009: 0",
    "Volatility for July 2009: 0",
    "Volatility for August 2009: 0",
    "Volatility for September 2009: 0",
    "Volatility for October 2009: 0",
    "Volatility for November 2009: 0",
    "Volatility for December 2010: 0",
    "Volatility for January 2010: 0",
    "Volatility for February 2010: 0",
    "Volatility for March 2010: 0",
    "Volatility for April 2010: 0",
    "Volatility for May 2010: 0",
    "Volatility for June 2010: 0",
    "Volatility for July 2010: 0",
    "Volatility for August 2010: 0",
    "Volatility for September 2010: 1",
    "Volatility for October 2010: 0",
    "Volatility for November 2010: 1",
    "Volatility for December 2011: 0",
    "Volatility for January 2011: 0",
    "Volatility for February 2011: 0",
    "Volatility for March 2011: 0",
    "Volatility for April 2011: 0",
    "Volatility for May 2011: 0",
    "Volatility for June 2011: 0",
    "Volatility for July 2011: 0",
    "Volatility for August 2011: 0",
    "Volatility for September 2011: 0",
    "Volatility for October 2011: 0",
    "Volatility for November 2011: 0",
    "Volatility for December 2012: 0"
]

# File path
file_path = r'C:\Users\mural\Downloads\monthlyvolatility.csv'

# Save data to CSV file
with open(file_path, mode='w', newline='') as file:
    writer = csv.writer(file)
    for item in data:
        writer.writerow([item])

print("Data has been saved to:", file_path)


Data has been saved to: C:\Users\mural\Downloads\monthlyvolatility.csv


In [36]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from statsmodels.tsa.seasonal import seasonal_decompose
import os
import numpy as np

In [37]:
data=pd.read_csv('NIFTY50_all.csv')

In [38]:
data['Date'] = pd.to_datetime(data['Date'])

# Calculation of SMA/EMA

In [41]:
def calculate_rolling_averages(df, window):
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['EMA'] = df['Close'].ewm(span=window, adjust=False).mean()
    return df

# to calculate averages annually
window_size = 20  

# rolling averages calculation
annual_averages = (
    data.groupby(data['Date'].dt.year)
    .apply(calculate_rolling_averages, window=window_size)
    .reset_index(drop=True)
)


for year, group in annual_averages.groupby(annual_averages['Date'].dt.year):
    print(f"\nYear: {year}")
    print(group[['Date', 'Close', 'SMA', 'EMA']])


Year: 2000
           Date   Close       SMA         EMA
0    2000-01-03  381.65       NaN  381.650000
1    2000-01-04  385.55       NaN  382.021429
2    2000-01-05  383.00       NaN  382.114626
3    2000-01-06  377.50       NaN  381.675138
4    2000-01-07  385.70       NaN  382.058458
...         ...     ...       ...         ...
7699 2000-12-22  261.70  299.3200  297.510152
7700 2000-12-26  258.55  297.8475  293.799661
7701 2000-12-27  269.05  296.8875  291.442551
7702 2000-12-28  277.85  296.4500  290.148022
7703 2000-12-29  277.80  296.2450  288.972020

[7704 rows x 4 columns]

Year: 2001
            Date   Close       SMA         EMA
7704  2001-01-01  272.20       NaN  272.200000
7705  2001-01-02  273.70       NaN  272.342857
7706  2001-01-03  275.05       NaN  272.600680
7707  2001-01-04  275.15       NaN  272.843473
7708  2001-01-05  268.85       NaN  272.463142
...          ...     ...       ...         ...
15563 2001-12-24  105.30  129.4450  121.218000
15564 2001-12-26  101.1

In [42]:
def calculate_rolling_averages(df, window):
    df['SMA'] = df['Close'].rolling(window=window).mean()
    df['EMA'] = df['Close'].ewm(span=window, adjust=False).mean()
    return df

# Set the window size for rolling averages
window_size = 20  

# Calculate rolling averages monthly
monthly_averages = (
    data.groupby([data['Date'].dt.year, data['Date'].dt.month])
    .apply(calculate_rolling_averages, window=window_size)
    .reset_index(drop=True)
)

for (year, month), group in monthly_averages.groupby([monthly_averages['Date'].dt.year, monthly_averages['Date'].dt.month]):
    print(f"\nYear: {year}, Month: {month}")
    print(group[['Date', 'Close', 'SMA', 'EMA']])


Year: 2000, Month: 1
          Date    Close        SMA          EMA
0   2000-01-03   381.65        NaN   381.650000
1   2000-01-04   385.55        NaN   382.021429
2   2000-01-05   383.00        NaN   382.114626
3   2000-01-06   377.50        NaN   381.675138
4   2000-01-07   385.70        NaN   382.058458
..         ...      ...        ...          ...
669 2000-01-24  1223.15  1632.5600  1434.781194
670 2000-01-25  1209.00  1518.6475  1413.278223
671 2000-01-27  1221.10  1391.3900  1394.975535
672 2000-01-28  1286.90  1252.3575  1384.682627
673 2000-01-31  1213.10  1122.7625  1368.341425

[674 rows x 4 columns]

Year: 2000, Month: 2
           Date    Close        SMA          EMA
674  2000-02-01   460.30        NaN   460.300000
675  2000-02-02   460.10        NaN   460.280952
676  2000-02-03   456.10        NaN   459.882766
677  2000-02-04   456.70        NaN   459.579646
678  2000-02-07   453.65        NaN   459.014918
...         ...      ...        ...          ...
1383 2000-02-

             Date   Close       SMA         EMA
152529 2014-07-01  244.20       NaN  244.200000
152530 2014-07-02  271.25       NaN  246.776190
152531 2014-07-03  270.40       NaN  249.026077
152532 2014-07-04  275.55       NaN  251.552165
152533 2014-07-07  280.70       NaN  254.328149
...           ...     ...       ...         ...
153602 2014-07-24  299.55  321.5850  335.142192
153603 2014-07-25  289.10  308.5025  330.757221
153604 2014-07-28  287.45  295.6550  326.632724
153605 2014-07-30  295.15  295.4375  323.634369
153606 2014-07-31  290.70  295.0750  320.497763

[1078 rows x 4 columns]

Year: 2014, Month: 8
             Date   Close       SMA         EMA
153607 2014-08-01  259.25       NaN  259.250000
153608 2014-08-04  265.45       NaN  259.840476
153609 2014-08-05  265.80       NaN  260.408050
153610 2014-08-06  268.50       NaN  261.178712
153611 2014-08-07  266.15       NaN  261.652168
...           ...     ...       ...         ...
154533 2014-08-22  289.10  349.8875  334.

# Market crash prediction using SMA/EMA

In [44]:
# Sudden drops in SMA and EMA indicate bearish market sentiment and potential downward pressure on prices
def detect_sudden_drop(df):
    # Calculate the difference between consecutive SMA and EMA values
    df['SMA_Diff'] = df['SMA'].diff()
    df['EMA_Diff'] = df['EMA'].diff()
    
    # Identify sudden drops where both SMA and EMA have negative differences
    sudden_drop = int((df['SMA_Diff'].iloc[-1] < 0) and (df['EMA_Diff'].iloc[-1] < 0))
    return pd.Series({'Sudden_Drop': sudden_drop})

# Group by year and month and apply the function
monthly_data = annual_averages.groupby([annual_averages['Date'].dt.year, annual_averages['Date'].dt.month]).apply(detect_sudden_drop)

# Print the data in the specified format
for index, row in monthly_data.iterrows():
    year, month = index
    sudden_drop = row['Sudden_Drop']
    print(f"{year} {month}: {sudden_drop}")


2000 1: 1
2000 2: 0
2000 3: 1
2000 4: 1
2000 5: 1
2000 6: 1
2000 7: 1
2000 8: 0
2000 9: 1
2000 10: 1
2000 11: 1
2000 12: 1
2001 1: 1
2001 2: 1
2001 3: 1
2001 4: 1
2001 5: 0
2001 6: 0
2001 7: 1
2001 8: 0
2001 9: 1
2001 10: 0
2001 11: 0
2001 12: 1
2002 1: 0
2002 2: 0
2002 3: 0
2002 4: 0
2002 5: 1
2002 6: 1
2002 7: 1
2002 8: 0
2002 9: 1
2002 10: 1
2002 11: 0
2002 12: 1
2003 1: 1
2003 2: 0
2003 3: 1
2003 4: 0
2003 5: 0
2003 6: 0
2003 7: 0
2003 8: 1
2003 9: 0
2003 10: 0
2003 11: 1
2003 12: 0
2004 1: 0
2004 2: 1
2004 3: 0
2004 4: 1
2004 5: 0
2004 6: 1
2004 7: 0
2004 8: 0
2004 9: 0
2004 10: 1
2004 11: 0
2004 12: 0
2005 1: 1
2005 2: 1
2005 3: 0
2005 4: 1
2005 5: 1
2005 6: 0
2005 7: 0
2005 8: 0
2005 9: 1
2005 10: 1
2005 11: 0
2005 12: 1
2006 1: 1
2006 2: 0
2006 3: 0
2006 4: 0
2006 5: 1
2006 6: 0
2006 7: 0
2006 8: 0
2006 9: 0
2006 10: 0
2006 11: 0
2006 12: 1
2007 1: 1
2007 2: 1
2007 3: 0
2007 4: 0
2007 5: 0
2007 6: 0
2007 7: 0
2007 8: 0
2007 9: 0
2007 10: 1
2007 11: 1
2007 12: 0
2008 1: 0
2008 2

In [47]:
data['Returns'] = (data['Close'] - data['Open']) / data['Open']

In [48]:
window_size = 20 
data['RollingAvg'] = data['Returns'].rolling(window=window_size).mean()

In [49]:
data['InvertedCurve'] = data['Returns'] > data['RollingAvg']

# INVERTED YIELD CURVE

In [51]:
data['Date'] = pd.to_datetime(data['Date'])
inverted_curve_points = data[data['InvertedCurve']]
for year, group in inverted_curve_points.groupby(data['Date'].dt.year):
    print(f'Year {year}:')
    print(group[['Date', 'Open', 'Close', 'RollingAvg']])

Year 2000:
             Date   Open   Close  RollingAvg
3322   2000-01-03  370.0  381.65    0.003647
3323   2000-01-04  380.0  385.55    0.002783
3324   2000-01-05  371.5  383.00    0.003614
3326   2000-01-07  376.0  385.70    0.001146
3333   2000-01-18  410.0  414.05    0.003198
...           ...    ...     ...         ...
230127 2000-12-18  295.0  301.60    0.001392
230129 2000-12-20  291.0  300.30    0.002268
230133 2000-12-27  260.0  269.05   -0.005869
230134 2000-12-28  274.5  277.85   -0.005539
230135 2000-12-29  277.2  277.80   -0.004714

[3761 rows x 4 columns]
Year 2001:
             Date    Open   Close  RollingAvg
3573   2001-01-02  270.05  273.70    0.007086
3574   2001-01-03  273.05  275.05    0.007006
3580   2001-01-11  261.35  269.00   -0.003490
3582   2001-01-15  249.00  265.15    0.001878
3583   2001-01-16  262.40  263.75   -0.002384
...           ...     ...     ...         ...
230371 2001-12-11  137.00  145.90    0.008020
230379 2001-12-24   98.00  105.30   -0.012570